# Neo4j & LLM Fundamentals
- https://graphacademy.neo4j.com/courses/llm-fundamentals/ 

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4


In [ ]:

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

llm = ChatOpenAI(openai_api_key='')

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="omoptograph",
    database="omoptograph"
)


Session ID: 495537fd-6238-4821-ae24-17836f2b897e


In [ ]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Neo4j expert having a conversation about how to create Cypher queries",
        ),
        ("human", "{input}"),
    ]
)

cypher_chat = prompt | llm | StrOutputParser()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

tools = [
    Tool.from_function(
        name="Cypher Support",
        description="For when you need to talk about Cypher queries.",
        func=cypher_chat.invoke,
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

cypher_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    q = input("> ")

    response = cypher_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

c:\Users\USER\Anaconda3\envs\nvidia_neo4j\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: LAST_MESSAGE)} {position: line: 1, column: 23, offset: 22} for query: 'MATCH (s:`Session`)-[:LAST_MESSAGE]->(last_message) WHERE s.id = $session_id MATCH p=(last_message)<-[:NEXT*0..6]-() WITH p, length(p) AS length ORDER BY length DESC LIMIT 1 UNWIND reverse(nodes(p)) AS node RETURN {data:{content: node.content}, type:node.type} AS result'
Receive

No, you do not need to use a tool at the moment. How can I assist you today?
No, you do not need to use a tool at the moment. How can I assist you today?
No, you do not need to use a tool at the moment. How can I assist you today?
No, you do not need to use a tool at the moment. How can I assist you today?
Thank you for providing the information on how to retrieve the number of patients with asthma in a Neo4j database.
How can I assist you today?
How can I assist you today?


## Initialising the LLM
- 241119
- https://graphacademy.neo4j.com/courses/llm-fundamentals/3-intro-to-langchain/2-initialising-the-llm/

In [ ]:
from langchain_openai import OpenAI

openai_api_key=""

llm = OpenAI(openai_api_key=openai_api_key)

response = llm.invoke("What is Neo4j?")

print(response)



Neo4j is a graph database management system designed to store, manage, and query highly connected data. It is based on the property graph model, where data is represented as nodes, relationships, and properties. Neo4j is a highly scalable, ACID-compliant database that uses a native graph storage and query engine to provide efficient and fast data retrieval. It is commonly used for applications that require complex data modeling, such as social networks, recommendation engines, fraud detection, and knowledge graphs. Neo4j also offers a variety of tools and libraries to help developers build applications on top of the database.


In [2]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(template="""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""", input_variables=["fruit"])

response = llm.invoke(template.format(fruit="apple"))

print(response)

"Well, mate, this 'ere is a nice little Adam and Eve, innit? A juicy, crisp little fruit that'll keep the doctor away. Want me to bag you up a couple of Bob Hope's?"


---
## Chains

In [5]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(openai_api_key=openai_api_key)

template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""")

llm_chain = template | llm

response = llm_chain.invoke({"fruit": "apple"})

print(response)


Well, luv, that's an Adam and Eve, innit? Crisp and juicy, it's a right treat. Can't go wrong with a nice, round apple.


In [6]:
from langchain.schema import StrOutputParser

llm_chain = template | llm | StrOutputParser()


template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Output JSON as {{"description": "your response here"}}

Tell me about the following fruit: {fruit}
""")

from langchain.output_parsers.json import SimpleJsonOutputParser

llm_chain = template | llm | SimpleJsonOutputParser()

---
## Chat MOdels

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage  

chat_llm = ChatOpenAI(
    openai_api_key=openai_api_key
)

instructions = SystemMessage(content="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.
""")

question = HumanMessage(content="What is the weather like?")

response = chat_llm.invoke([
    instructions,
    question
])

print(response.content)

Dude, the weather's totally gnarly! The sun's shining, the waves are firing, and the offshore winds are epic! It's gonna be a sick day out there, for sure!


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        (
            "human", 
            "{question}"
        ),
    ]
)

chat_chain = prompt | chat_llm | StrOutputParser()

response = chat_chain.invoke({"question": "What is the weather like?"})

print(response)

Dude, the weather is totally gnarly today! We’ve got some sick waves rolling in, perfect for shredding. Grab your board and let’s catch some barrels!


In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# chat_llm = ChatOpenAI(openai_api_key="sk-...")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ( "system", "{context}" ),
        ( "human", "{question}" ),
    ]
)

chat_chain = prompt | chat_llm | StrOutputParser()

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

response = chat_chain.invoke(
    {
        "context": current_weather,
        "question": "What is the weather like on Watergate Bay?",
    }
)

print(response)

Dude, Watergate Bay is firing right now! We got some sick 3ft waves rolling in, but watch out for those onshore winds messing with the lineup. It's gonna be a gnarly session out there!


---
## Giving a Chat Model Memory

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

memory = ChatMessageHistory()

def get_memory(session_id):
    return memory

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)


current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while True:
    question = input("> ")

    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": "none"}
        }
    )
    
    print(response)

Dude, the LLM stands for the local lineup, where we check out the sick waves at different beaches! So, what's the word on the surf at Fistral, Bells, and Watergate Bay today?
RAG stands for "radical and gnarly," bro! It's like when the waves are firing and the surf is totally awesome. Unfortunately, it doesn't sound like the surf is RAG at Bells with those flat and calm conditions. But Fistral is looking killer with 6ft waves and offshore winds, and Watergate Bay has 3ft waves with onshore winds. Time to catch some waves, dude!
Oh, I got you, man! RAG could be like "Riding All the Greatness" in the Local Lineup, where we score epic waves and have an awesome time shredding the surf together. Stoked to hit up Fistral and Watergate Bay today, those spots are going to be RAG for sure!
Dude, I'm all about surfing and catching waves, so I'm not familiar with retriever augment generation. But if you ever want to talk surf conditions, beach vibes, or anything gnarly like that, I'm your guy! Le

KeyboardInterrupt: Interrupted by user

---
## Storing Conversation History

In [12]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="omoptograph",
    database="omoptograph"
)

result = graph.query("""
MATCH (p:Person) 
RETURN p.person_id
""")

print(result)

[{'p.person_id': '520739'}, {'p.person_id': '1014200'}, {'p.person_id': '412875'}, {'p.person_id': '1255443'}, {'p.person_id': '1060907'}, {'p.person_id': '1378010'}, {'p.person_id': '855706'}, {'p.person_id': '1375743'}, {'p.person_id': '284048'}, {'p.person_id': '1192619'}, {'p.person_id': '45104'}, {'p.person_id': '1176092'}, {'p.person_id': '1351165'}, {'p.person_id': '1423384'}, {'p.person_id': '932868'}, {'p.person_id': '889739'}, {'p.person_id': '949765'}, {'p.person_id': '1451159'}, {'p.person_id': '430337'}, {'p.person_id': '120956'}, {'p.person_id': '62216'}, {'p.person_id': '869402'}, {'p.person_id': '1163284'}, {'p.person_id': '316046'}, {'p.person_id': '1425297'}, {'p.person_id': '164949'}, {'p.person_id': '136903'}, {'p.person_id': '723749'}, {'p.person_id': '856472'}, {'p.person_id': '668304'}, {'p.person_id': '356317'}, {'p.person_id': '1084067'}, {'p.person_id': '351281'}, {'p.person_id': '593441'}, {'p.person_id': '432761'}, {'p.person_id': '1445923'}, {'p.person_id':

In [13]:
print(graph.schema)

Node properties:
Ethnicity {ethnicity_concept_name: STRING, ethnicity_concept_id: STRING}
Measurement {measurement_concept_name: STRING, measurement_date: STRING, measurement_concept_id: STRING, measurement_id: STRING}
Race {race_concept_name: STRING, race_concept_id: STRING}
Observation {observation_concept_name: STRING, observation_date: STRING, observation_concept_id: STRING, observation_id: STRING}
ConditionOccurrence {condition_concept_name: STRING, condition_concept_id: STRING, condition_start_date: STRING, condition_end_date: STRING, condition_occurrence_id: STRING}
ProcedureOccurrence {procedure_concept_name: STRING, procedure_date: STRING, procedure_occurrence_id: STRING}
DrugExposure {drug_concept_name: STRING, drug_concept_id: STRING, drug_exposure_start_date: STRING, drug_exposure_end_date: STRING, drug_type_concept_id: STRING, drug_type_concept_name: STRING, drug_exposure_id: STRING}
ObservationPeriod {period_type_concept_name: STRING, period_type_concept_id: STRING, obser

In [14]:
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: 5c693d9a-333c-4589-9011-8f4e9ef67989


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="omoptograph",
    database="omoptograph"
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while True:
    question = input("> ")

    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": SESSION_ID}
        }
    )
    
    print(response)

Session ID: dcc2dee0-ad52-4a72-9908-c022bc582d2b
Dude, Bells is totally flat and calm right now. Not even a ripple to ride, bummer!
Cowabunga, catch you later, surfer friend!
Stoked to chat about the surf with you, dude! Catch you on the flip side!
Hang loose, bro! Catch you on the next wave!


---
## Agents

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")
openai_api_key = ''
llm = ChatOpenAI(openai_api_key=openai_api_key)

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="omoptograph"
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_iterations=3,
    verbose=True,
    handle_parse_errors=True
)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    q = input("> ")

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

Session ID: ed05dae9-19fb-4e4e-8fd4-1734218c7b76


c:\Users\USER\Anaconda3\envs\nvidia_neo4j\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


No, you do not need to use a tool at the moment. How can I assist you today?
Hi! I am Assistant, a large language model trained by OpenAI. How can I assist you today?
Of course! What genre or type of movie are you in the mood for? Let me know your preferences, and I'll recommend some movies for you.
Please specify the genre or plot you are interested in, such as action, comedy, romance, horror, science fiction, etc. Let me know your preferences so I can recommend some movies for you!
I hope you find these action movie recommendations enjoyable! If you have any specific preferences or need more recommendations, feel free to let me know.
Here are some popular romance movies that you may enjoy:

1. "The Notebook" (2004)
2. "Titanic" (1997)
3. "La La Land" (2016)
4. "Pride and Prejudice" (2005)
5. "Love, Simon" (2018)
6. "Crazy, Stupid, Love" (2011)
7. "Before Sunrise" (1995)
8. "The Fault in Our Stars" (2014)
9. "The Princess Bride" (1987)
10. "Eternal Sunshine of the Spotless Mind" (2004

KeyboardInterrupt: 

In [3]:
from langchain_community.tools import YouTubeSearchTool

youtube = YouTubeSearchTool()

In [4]:
def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

In [ ]:
tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

In [ ]:
tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

In [5]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

llm = ChatOpenAI(openai_api_key=openai_api_key)

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="omoptograph",
    database="omoptograph"
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

youtube = YouTubeSearchTool()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    q = input("> ")

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

Session ID: 5377c991-bdd1-45f2-a1df-dcc03d245f11


c:\Users\USER\Anaconda3\envs\nvidia_neo4j\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Here are the links to the movie trailer for "The Matrix":
1. [The Matrix - Official Trailer #1](https://www.youtube.com/watch?v=8DajVKAkL50&pp=ygUKVGhlIE1hdHJpeA%3D%3D)
2. [The Matrix - Official Trailer #2](https://www.youtube.com/watch?v=Z2eCmhBgsyI&pp=ygUKVGhlIE1hdHJpeA%3D%3D)
I have found the trailers for you. Here are the links:
1. [Trailer 1](https://www.youtube.com/watch?v=jdv_PMffxyE&pp=ygUYQ2FuIHlvdSBmaW5kIHRoZSB0cmFpbGVy)
2. [Trailer 2](https://www.youtube.com/watch?v=hKDSGy_FsBI&pp=ygUYQ2FuIHlvdSBmaW5kIHRoZSB0cmFpbGVy)


KeyboardInterrupt: 

---
## Retirevers

In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

embedding_provider = OpenAIEmbeddings(
    openai_api_key=openai_api_key
)

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="omoptograph",
    database="omoptograph"
)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="ConditionOccurrence",
    embedding_node_property="plotEmbedding",
    text_node_property="plot",
)

result = movie_plot_vector.similarity_search("A gastritis's complications.")
for doc in result:
    print(doc.metadata["condition_concept_name"], "-", doc.page_content)

ValueError: The specified vector index name does not exist. Make sure to check if you spelled it correctly

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector


openai_api_key = ''

embedding_provider = OpenAIEmbeddings(
    openai_api_key=openai_api_key
)

graph = Neo4jGraph(
    url="bolt://54.90.71.146:7687",
    username="neo4j",
    password="tractor-locomotive-tables"
)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="moviePlots",
    embedding_node_property="plotEmbedding",
    text_node_property="plot",
)

result = movie_plot_vector.similarity_search("A movie where aliens land and attack earth.")
for doc in result:
    print(doc.metadata["title"], "-", doc.page_content)

Coneheads - Aliens with conical crania crash land on Earth.
Aliens - The planet from Alien (1979) has been colonized, but contact is lost. This time, the rescue team has impressive firepower, but will it be enough?
Independence Day (a.k.a. ID4) - The aliens are coming and their goal is to invade and destroy Earth. Fighting superior technology, mankind's best weapon is the will to survive.
Arrival, The - Zane, an astronomer, discovers intelligent alien life. But the aliens are keeping a deadly secret, and will do anything to stop Zane from learning it.


---
## Adding the Neo4j Vector Retriever

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

llm = ChatOpenAI(openai_api_key=openai_api_key)

# graph = Neo4jGraph(
#     url="bolt://localhost:7687",
#     username="neo4j",
#     password="pleaseletmein"
# )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

youtube = YouTubeSearchTool()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    q = input("> ")

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

Session ID: d1320998-3459-4373-8977-31ad08a72c74


c:\Users\USER\Anaconda3\envs\nvidia_neo4j\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: LAST_MESSAGE)} {position: line: 1, column: 23, offset: 22} for query: 'MATCH (s:`Session`)-[:LAST_MESSAGE]->(last_message) WHERE s.id = $session_id MATCH p=(last_message)<-[:NEXT*0..6]-() WITH p, length(p) AS length ORDER BY length DESC LIMIT 1 UNWIND reverse(nodes(p)) AS node RETURN {data:{content: node.content}, type:node.type} AS result'
Receive

I am Assistant, a large language model trained by OpenAI. I am here to assist you with a wide range of tasks and provide information on various topics. How can I help you today?
One romantic movie that I would recommend is "The Notebook." It is a beautiful and heartwarming tale of enduring love that transcends time and circumstances.
I would like a recommendation for a comedy movie, please.
Sure, feel free to ask me any questions or let me know how I can assist you further.
I recommend the movie "Superbad" for a comedy movie recommendation. It's a hilarious coming-of-age comedy with plenty of laugh-out-loud moments. Enjoy!


KeyboardInterrupt: Interrupted by user

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector


llm = ChatOpenAI(openai_api_key=openai_api_key)

embedding_provider = OpenAIEmbeddings(openai_api_key=openai_api_key)

# graph = Neo4jGraph(
#     url="bolt://localhost:7687",
#     username="neo4j",
#     password="pleaseletmein"
# )

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="moviePlots",
    embedding_node_property="plotEmbedding",
    text_node_property="plot",
)

plot_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=movie_plot_vector.as_retriever()
)

response = plot_retriever.invoke(
    {"query": "A movie where a mission to the moon goes wrong"}
)

print(response)

{'query': 'A movie where a mission to the moon goes wrong', 'result': 'The movie you are looking for is "2001: A Space Odyssey" directed by Stanley Kubrick. In this film, a mission to the moon goes wrong when a mysterious artificial object is discovered buried beneath the Lunar surface, leading to unexpected consequences.'}


In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

OPENAI_API_KEY = openai_api_key

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

embedding_provider = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# graph = Neo4jGraph(
#     url="bolt://localhost:7687",
#     username="neo4j",
#     password="pleaseletmein"
# )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

youtube = YouTubeSearchTool()

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="moviePlots",
    embedding_node_property="plotEmbedding",
    text_node_property="plot",
)

plot_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=movie_plot_vector.as_retriever()
)

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
    Tool.from_function(
        name="Movie Plot Search",
        description="For when you need to compare a plot to a movie. The question will be a string. Return a string.",
        func=plot_retriever.invoke,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    q = input("> ")

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

Session ID: 31b4665e-5562-496f-95ee-67e165155582


c:\Users\USER\Anaconda3\envs\nvidia_neo4j\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Here is a YouTube clip related to The Notebook movie: [https://www.youtube.com/watch?v=BjJcYdEOI0k&pp=ygUaVGhlIE5vdGVib29rIG1vdmllIHRyYWlsZXI%3D]
The plot you described sounds like the storyline of the movie "Titanic" starring Leonardo DiCaprio and Kate Winslet.
The plot of "The Notebook" movie follows the love story of a young couple in the 1940s, separated by social class differences and World War II, who reconnect years later in their old age. On the other hand, "Titanic" is about a young couple from different social classes who fall in love aboard the ill-fated RMS Titanic. Both movies involve passionate love stories and obstacles that the couples must overcome, but "The Notebook" focuses on a lifetime of love and commitment, while "Titanic" is set against the backdrop of a historical tragedy.
Of course! Romantic movies are always a favorite for many people. There are so many classic romantic movies as well as newer ones that have captured hearts around the world. Do you have a fav

KeyboardInterrupt: Interrupted by user

---
## Using LLMs for Query Generation
241120

---
## The Cypher QA Chain

In [11]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    openai_api_key=openai_api_key
)

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="omoptograph",
    database="omoptograph"
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)


In [13]:

cypher_chain.invoke({"query": "What is the most frequent condition name?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:ConditionOccurrence)
RETURN c.condition_concept_name, COUNT(c) AS frequency
ORDER BY frequency DESC
LIMIT 1;
Full Context:
[{'c.condition_concept_name': 'Gastritis', 'frequency': 551}]

> Finished chain.


{'query': 'What is the most frequent condition name?',
 'result': 'Gastritis is the most frequent condition name.'}

In [15]:
cypher_chain.invoke({"query": "I want to know every unique concept_id of Gatritis"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
To find every unique concept_id of Gatritis, you can use the following Cypher query:

MATCH (co:ConditionOccurrence)
WHERE co.condition_concept_name = 'Gatritis'
RETURN DISTINCT co.condition_concept_id


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'To': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"To find every unique concept_id of Gatritis, you can use the following Cypher query:"
 ^}

---
## Providing Specific Instructions

In [16]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
cypher_chain.invoke({"query": "Gatritis으로 진단받은 사람들의 처방받은 약물들에서 성별에 따른 상위 20개의 약물들을 카운트해서 나열해 주는데 카운트는 약물의 처방수로 해줘"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:HAS_CONDITION_OCCURRENCE]->(c:ConditionOccurrence)
WHERE c.condition_concept_name = 'Gastritis'
MATCH (p)-[:HAS_DRUG_EXPOSURE]->(d:DrugExposure)
WITH d, p
MATCH (p)-[:HAS_Gender]->(g:Gender)
RETURN d.drug_concept_name AS DrugName, g.gender_concept_name AS Gender, COUNT(d) AS PrescriptionCount
ORDER BY COUNT(d) DESC
LIMIT 20;
Full Context:
[{'DrugName': 'almagate 500 MG Oral Tablet', 'Gender': 'FEMALE', 'PrescriptionCount': 20238}, {'DrugName': 'bepotastine 7.11000000000000032 MG Oral Tablet', 'Gender': 'FEMALE', 'PrescriptionCount': 19702}, {'DrugName': 'Streptodornase 2500 UNT / Streptokinase 10000 UNT Oral Tablet', 'Gender': 'FEMALE', 'PrescriptionCount': 18523}, {'DrugName': 'erdosteine 300 MG Oral Capsule', 'Gender': 'FEMALE', 'PrescriptionCount': 16147}, {'DrugName': 'Cefaclor 250 MG Oral Capsule', 'Gender': 'FEMALE', 'PrescriptionCount': 10865}, {'DrugName': 'montelukast 10 MG Oral Tablet', 'Gender'

In [38]:
question = "Asthma 진단받은 사람들의 약물 처방 패턴이 어떻게 되는지 남여 차이가 있는지 확인해줘"

In [39]:
cypher_chain.invoke({"query": question})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:HAS_CONDITION_OCCURRENCE]->(co:ConditionOccurrence)
WHERE co.condition_concept_name = "Asthma"
MATCH (p)-[:HAS_DRUG_EXPOSURE]->(de:DrugExposure)
WITH p, de
MATCH (p)-[:HAS_Gender]->(g:Gender)
RETURN g.gender_concept_name, de.drug_concept_name, COUNT(de) as prescription_count
ORDER BY g.gender_concept_name, prescription_count DESC
Full Context:
[{'g.gender_concept_name': 'FEMALE', 'de.drug_concept_name': 'almagate 500 MG Oral Tablet', 'prescription_count': 10624}, {'g.gender_concept_name': 'FEMALE', 'de.drug_concept_name': 'bepotastine 7.11000000000000032 MG Oral Tablet', 'prescription_count': 10371}, {'g.gender_concept_name': 'FEMALE', 'de.drug_concept_name': 'Streptodornase 2500 UNT / Streptokinase 10000 UNT Oral Tablet', 'prescription_count': 9542}, {'g.gender_concept_name': 'FEMALE', 'de.drug_concept_name': 'erdosteine 300 MG Oral Capsule', 'prescription_count': 8427}, {'g.gender_concept_name': 'FEMALE

{'query': 'Asthma 진단받은 사람들의 약물 처방 패턴이 어떻게 되는지 남여 차이가 있는지 확인해줘',
 'result': '남여 차이를 확인하기 위해 여성 환자들이 받은 처방 패턴을 살펴보면, 알마게이트 500 MG 경구정, 베포타스틴 7.11 MG 경구정, 스트렙토도르네이즈 2500 UNT / 스트렙토키나제 10000 UNT 경구정, 에르도스테인 300 MG 경구캡슐, 세파클로르 250 MG 경구캡슐 등이 포함됩니다. 남성 환자들의 데이터가 제공되지 않아 정확한 남여 차이를 확인할 수 없습니다.'}

---
## Customization
- START: 241120

In [62]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer specializing in OMOP-CDM data models and graph database queries. 
Your role is to translate user questions into Cypher queries to extract meaningful insights from the OMOP-CDM-based Neo4j graph database.

Instructions:
1. Use only the provided node labels, relationship types, and properties from the schema.
2. Do not invent or assume any additional labels, relationships, or properties.
3. Ensure the query captures the full intent of the user's question while adhering strictly to the schema.
4. Prioritize efficient graph traversal using Neo4j best practices.

Schema: {schema}
Question: {question}

Output a Cypher query that accurately answers the user's question based on the provided schema.
"""

schema = """
Nodes:
- Person: {id: STRING, gender: STRING, birth_date: DATE}
- ConditionOccurrence: {condition_id: STRING, start_date: DATE, end_date: DATE}
- DrugExposure: {drug_id: STRING, start_date: DATE, end_date: DATE}
- ProcedureOccurrence: {procedure_id: STRING, date: DATE}
- VisitOccurrence: {visit_id: STRING, start_date: DATE, end_date: DATE}
- ObservationPeriod: {period_id: STRING, start_date: DATE, end_date: DATE}

Relationships:
- (:Person)-[:HAS_CONDITION_OCCURRENCE]->(:ConditionOccurrence)
- (:Person)-[:HAS_DRUG_EXPOSURE]->(:DrugExposure)
- (:Person)-[:HAS_PROCEDURE_OCCURRENCE]->(:ProcedureOccurrence)
- (:Person)-[:HAS_VISIT_OCCURRENCE]->(:VisitOccurrence)
- (:Person)-[:HAS_OBSERVATION_PERIOD]->(:ObservationPeriod)
- (:VisitOccurrence)-[:ASSOCIATED_DURING_VISIT]->(:ConditionOccurrence)
- (:VisitOccurrence)-[:ASSOCIATED_DURING_VISIT]->(:DrugExposure)
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [86]:
question1 = "Asthma를 진단받은 환자에게 처방된 약물 중 가장 많이 처방 받은 약물 어떤것인지 확인해줘"

In [ ]:
question2 = "Asthma를 진단받은 환자에게 처방된 약물 중 가장 많이 처방 받은 약물과 함께 처방된 다른 약물들이 어떤것인지 확인해줘"

In [88]:
question3 = "Asthma를 진단받은 환자에게 처방된 약물 중 가장 많이 처방 받은 약물과 함께 처방된 다른 약물들 중 가장 빈번한 약물이 어떤것인지 확인해줘"

In [90]:
cypher_chain.invoke({"query": question3})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:HAS_CONDITION_OCCURRENCE]->(c:ConditionOccurrence {condition_concept_name: "Asthma"})
MATCH (p)-[:HAS_DRUG_EXPOSURE]->(d1:DrugExposure)
WHERE NOT d1.drug_concept_name = "Albuterol" // Exclude the most prescribed drug (Albuterol)
MATCH (d1)-[:ASSOCIATED_DURING_VISIT]->(:VisitOccurrence)<-[:ASSOCIATED_DURING_VISIT]-(d2:DrugExposure)
RETURN d2.drug_concept_name, COUNT(*) AS frequency
ORDER BY frequency DESC
LIMIT 1;
Full Context:
[{'d2.drug_concept_name': 'almagate 500 MG Oral Tablet', 'frequency': 42453}]

> Finished chain.


{'query': 'Asthma를 진단받은 환자에게 처방된 약물 중 가장 많이 처방 받은 약물과 함께 처방된 다른 약물들 중 가장 빈번한 약물이 어떤것인지 확인해줘',
 'result': 'almagate 500 MG Oral Tablet이 가장 많이 처방된 약물이며, 다른 약물들 중에서도 가장 빈번하게 처방된 약물입니다.'}

In [ ]:
question4 = "Asthma를 진단받은 환자의 동반질환에 대해 통계값 산출해줘"

In [95]:
cypher_chain.invoke({"query": question4})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:HAS_CONDITION_OCCURRENCE]->(c:ConditionOccurrence)
WHERE c.condition_concept_name = "Asthma"
MATCH (p)-[:HAS_CONDITION_OCCURRENCE]->(co:ConditionOccurrence)
WHERE co.condition_concept_name <> "Asthma"
RETURN co.condition_concept_name, COUNT(co) as occurrence_count
ORDER BY occurrence_count DESC;
Full Context:
[{'co.condition_concept_name': 'Gastritis', 'occurrence_count': 7178}, {'co.condition_concept_name': 'Allergic rhinitis', 'occurrence_count': 5443}, {'co.condition_concept_name': 'Seasonal allergic rhinitis', 'occurrence_count': 5441}, {'co.condition_concept_name': 'Chronic serous otitis media', 'occurrence_count': 4453}, {'co.condition_concept_name': 'Heartburn', 'occurrence_count': 3173}, {'co.condition_concept_name': 'Acute bronchitis', 'occurrence_count': 3089}, {'co.condition_concept_name': 'Bronchitis', 'occurrence_count': 1288}, {'co.condition_concept_name': 'Acute suppurative otitis media', '

{'query': 'Asthma를 진단받은 환자의 동반질환에 대해 통계값 산출해줘',
 'result': 'Gastritis: 7178, Allergic rhinitis: 5443, Seasonal allergic rhinitis: 5441, Chronic serous otitis media: 4453, Heartburn: 3173, Acute bronchitis: 3089, Bronchitis: 1288, Acute suppurative otitis media: 864, Acute maxillary sinusitis: 614, Acute laryngotracheitis: 556'}

In [96]:
question5 = "Asthma를 진단받은 환자의 동반질환에 대해 통계값 산출하는데 중복제거한 환자 기준으로 계산해줘"

In [97]:
cypher_chain.invoke({"query": question5})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:HAS_CONDITION_OCCURRENCE]->(c:ConditionOccurrence)
WHERE c.condition_concept_name = "Asthma"
WITH DISTINCT p
MATCH (p)-[:HAS_CONDITION_OCCURRENCE]->(co:ConditionOccurrence)
WHERE co.condition_concept_name <> "Asthma"
RETURN co.condition_concept_name, COUNT(DISTINCT p) as num_patients_with_condition
ORDER BY num_patients_with_condition DESC;
Full Context:
[{'co.condition_concept_name': 'Acute bronchitis', 'num_patients_with_condition': 16}, {'co.condition_concept_name': 'Allergic rhinitis', 'num_patients_with_condition': 15}, {'co.condition_concept_name': 'Acute pharyngitis', 'num_patients_with_condition': 10}, {'co.condition_concept_name': 'Gastritis', 'num_patients_with_condition': 9}, {'co.condition_concept_name': 'Inflammatory disorder of digestive tract', 'num_patients_with_condition': 8}, {'co.condition_concept_name': 'Gastro-esophageal reflux disease with esophagitis', 'num_patients_with_condition':

{'query': 'Asthma를 진단받은 환자의 동반질환에 대해 통계값 산출하는데 중복제거한 환자 기준으로 계산해줘',
 'result': '통계를 내기 위해 중복되는 환자를 제외한 경우, 환자의 동반 질환은 다음과 같습니다: 급성 기관지염, 알레르기성 비염, 급성 인후염, 위염, 소화기의 염증성 질환, 식도염이 동반된 위식도 역류병, 탈구/긴장/긴장, 급성 위염, 근육통.'}